<a href="https://colab.research.google.com/github/harshitha-0531/Predicting_future_stores_sales/blob/main/day6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, r2_score
sns.set_style('whitegrid')
plt.rcParams['figure.figsize']=(12,8)


In [ ]:
!git clone https://github.com/GeeksforgeeksDS/21-Days-21-Projects-Dataset.git


In [ ]:
df= pd.read_csv('/content/21-Days-21-Projects-Dataset/Datasets/airline_passenger_timeseries.csv')
df.head(5)

In [ ]:
df.info()
df.describe()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['Month'], df['Passengers'])
plt.xlabel('Month')
plt.ylabel('Number of Passengers')
plt.title('Airline Passengers Over Time')
plt.xticks(df['Month'][::12]) # Show only yearly ticks for clarity
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd

# Load the dataframe
df= pd.read_csv('/content/21-Days-21-Projects-Dataset/Datasets/airline_passenger_timeseries.csv')

df['Month'] = pd.to_datetime(df['Month']) # Access the 'Month' column
df.set_index('Month', inplace=True) # Re-set the index (optional, but good practice)


decomposition = seasonal_decompose(df['Passengers'], model='additive')
decomposition.plot()
plt.show()

In [ ]:
def test_stationarity(timeseries):
  print("Result of Dickey-Fuller test:")
  dftest= adfuller(timeseries, autolag='AIC')
  dfoutput= pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
  for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)' %key]= value
  print(dfoutput)
test_stationarity(df['Passengers'])


In [ ]:
df_log=np.log(df['Passengers'])
df_diff=df_log.diff().dropna()
df_diff.plot()
plt.title('Logarithmic Transformation')
plt.show()
test_stationarity(df_log)
test_stationarity(df_diff)

In [ ]:
fig,(ax1,ax2)= plt.subplots(2,1, figsize=(12,8))
plot_acf(df_diff)
plot_pacf(df_diff)
plt.show()


In [ ]:
# split data into taining and test sets
train_data =df_log[:'1958']
test_data =df_log['1959':]
model=ARIMA(train_data,order=(1,1,1,),freq='MS')
arima_res=model.fit()
forecast=arima_res.get_forecast(steps=len(test_data))
forecast_ci=forecast.conf_int()

plt.figure(figsize=(14, 7))
plt.plot(df_log, label=' Log Data')
plt.plot(forecast.predicted_mean, label='Forecast')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=.15)
plt.title('Arima Forecast')
plt.legend()
plt.show()

In [ ]:
sarima_model=sm.tsa.statespace.SARIMAX(train_data,
                                       order=(1,1,1),
                                       seasonal_order=(1,1,1,12),
                                       enforce_stationarity=False,
                                       enforce_invertibility=False,
                                       freq='MS')
sarima_res=sarima_model.fit()
sarima_forecast=sarima_res.get_forecast(steps=len(test_data))
sarima_forecast_ci=sarima_forecast.conf_int()
plt.figure(figsize=(14, 7))
plt.plot(df_log, label=' Log Data')
plt.plot(sarima_forecast.predicted_mean, label='SARIMA Forecast', color='red')
plt.fill_between(sarima_forecast_ci.index, sarima_forecast_ci.iloc[:, 0], sarima_forecast_ci.iloc[:, 1], color='r', alpha=.15)
plt.title('Sarima Forecast')
plt.legend()
plt.show()

In [ ]:
original_test_data=np.exp(test_data)
sarima_predict=np.exp(sarima_forecast.predicted_mean)
rmse=np.sqrt(mean_squared_error(original_test_data,sarima_predict))
print(f"sarima model RSME:{rmse:.2f}")
plt.figure(figsize=(14,7))
plt.plot(df['Passengers'],label='Original Data')
plt.plot(sarima_predict,label='SARIMA FORECAST',color='red')
plt.title('SARIMA Forecast vs Original Data')
plt.legend()
plt.show()